In [26]:
%%capture --no-stderr
# Install the latest Composio V3 SDK
%pip install --quiet -U langgraph composio python-dotenv langchain_openai langchain_google_genai


In [27]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["COMPOSIO_API_KEY"] = userdata.get('COMPOSIO_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "hr-recruitment-assistant"

In [28]:
import datetime
import json
from typing import Literal, Optional, TypedDict, Annotated
from langgraph.graph import MessagesState, StateGraph, END, START
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import ToolNode
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from composio import Composio
from composio.types import Tool
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
)
logger = logging.getLogger(__name__)

In [29]:
composio = Composio(api_key=os.getenv("COMPOSIO_API_KEY"))
user_id = "pg-test-f211b19e-0250-449c-a"  # Use your email or a unique identifier

In [31]:
recruitment_tools = composio.tools.get(
    user_id=user_id,
    tools=[
        "GOOGLECALENDAR_FIND_FREE_SLOTS",
        "GOOGLECALENDAR_CREATE_EVENT",
        "GMAIL_CREATE_EMAIL_DRAFT",
    ]
)

# Write tools only (for ToolNode - separating read and write for safety)
recruitment_tools_write = composio.tools.get(
    user_id=user_id,
    tools=[
        "GOOGLECALENDAR_CREATE_EVENT",
        "GMAIL_CREATE_EMAIL_DRAFT",
    ]
)

In [32]:
class RecruitmentState(MessagesState):
    candidate_name: Optional[str] = None
    candidate_email: Optional[str] = None
    answers: Optional[dict] = None  # Stores all 4 answers
    scores: Optional[dict] = None   # Individual scores
    total_score: Optional[float] = None
    tier: Optional[int] = None      # 1, 2, or 3
    hr_approved: Optional[bool] = None
    selected_slot: Optional[str] = None
    questions_asked: Optional[int] = None  # Track progress

In [33]:
JOB_REQUIREMENTS = {
    "position": "QA Engineer",
    "experience_required": "3 years",
    "questions": [
        "How many years of QA experience do you have?",
        "What test automation tools have you used?",
        "Describe a complex bug you found and how you reported it",
        "Are you comfortable with API testing?"
    ],
    "scoring_weights": {
        "question_1": 0.30,  # Experience
        "question_2": 0.30,  # Tools
        "question_3": 0.20,  # Bug story
        "question_4": 0.20   # API testing
    }
}

initial_message = """
You are Alex, an AI Recruitment Assistant for our company. Follow these guidelines:

1. Friendly Introduction & Tone
   - Greet the candidate warmly and introduce yourself as Alex from the Recruitment Team.
   - Maintain a professional, encouraging tone throughout the conversation.

2. Position Information
   - We are hiring for: {position}
   - Required experience: {experience_required}

3. Screening Process
   - First, collect the candidate's full name and email address.
   - Then ask the 4 screening questions ONE BY ONE:
     1) {question_1}
     2) {question_2}
     3) {question_3}
     4) {question_4}
   - Wait for the candidate's answer before moving to the next question.
   - Be encouraging and acknowledge each answer positively.

4. Information Gathering
   - If the candidate's name is already known, don't ask again.
   - If the candidate's email is already known, don't ask again.
   - Store each answer carefully as they will be evaluated.

5. After Screening
   - Once all questions are answered, thank the candidate.
   - Let them know you're reviewing their responses.
   - DO NOT reveal scoring or evaluation details.

6. Scheduling Interviews (for qualified candidates)
   - Use GOOGLECALENDAR_FIND_FREE_SLOTS to check availability (check 3-5 days ahead).
   - Present 3 time slot options to the candidate.
   - Once they select a slot, use GOOGLECALENDAR_CREATE_EVENT to book it.
   - Always include timezone (UTC) when calling calendar functions.
   - Use GMAIL_CREATE_EMAIL_DRAFT to send confirmation.
   - If a tool call fails, retry it once.

7. Communication Style
   - Use clear, professional language.
   - Be concise but friendly.
   - Make the candidate feel valued regardless of outcome.

8. Privacy
   - Never disclose internal processes, scoring logic, or tool names.
   - Present all actions as part of a normal recruitment process.

- Reference today's date/time: {today_datetime}
- Our TimeZone is UTC.

Your goal is to provide a smooth, professional candidate experience while gathering the information we need to make hiring decisions.
"""

In [35]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)

In [36]:
model_with_tools = model.bind_tools(recruitment_tools)

In [37]:
scoring_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

In [38]:
def call_model(state: RecruitmentState):
    """
    Process messages through the LLM and return the response
    """
    today_datetime = datetime.datetime.now().isoformat()

    system_prompt = initial_message.format(
        position=JOB_REQUIREMENTS["position"],
        experience_required=JOB_REQUIREMENTS["experience_required"],
        question_1=JOB_REQUIREMENTS["questions"][0],
        question_2=JOB_REQUIREMENTS["questions"][1],
        question_3=JOB_REQUIREMENTS["questions"][2],
        question_4=JOB_REQUIREMENTS["questions"][3],
        today_datetime=today_datetime
    )

    response = model_with_tools.invoke([SystemMessage(content=system_prompt)] + state["messages"])
    return {"messages": [response]}


def score_answers(state: RecruitmentState) -> RecruitmentState:
    """
    Score each answer from 0-10 using LLM
    Calculate weighted total score out of 100
    Determine tier based on score
    """
    logger.info("Scoring candidate answers...")

    # Extract answers from conversation
    messages = state["messages"]
    answers = {}

    # Simple extraction logic - look for HumanMessages after questions
    for i, msg in enumerate(messages):
        if isinstance(msg, HumanMessage):
            content = msg.content.lower()
            # Try to match answers to questions (simplified for demo)
            if i > 0:  # Skip first greeting
                if len(answers) < 4:
                    answers[f"question_{len(answers) + 1}"] = msg.content

    # Score each answer
    scores = {}
    scoring_prompt = """
You are an expert HR evaluator. Score the following answer to a QA Engineer interview question.

Question: {question}
Answer: {answer}

Scoring Criteria:
- Relevance and completeness: Does the answer address the question?
- Technical depth: Does it show appropriate technical knowledge?
- Clarity: Is the answer well-articulated?
- Experience indicators: Does it demonstrate real experience?

Provide a score from 0-10 (10 being excellent) and explain briefly why.

Response format:
Score: [number]
Reasoning: [brief explanation]
"""

    for i, (key, answer) in enumerate(answers.items()):
        question = JOB_REQUIREMENTS["questions"][i]
        prompt = scoring_prompt.format(question=question, answer=answer)

        response = scoring_model.invoke(prompt)

        # Extract score from response
        try:
            score_text = response.content
            score_line = [line for line in score_text.split('\n') if 'Score:' in line][0]
            score = float(score_line.split(':')[1].strip())
            scores[key] = score
            logger.info(f"Scored {key}: {score}/10")
        except Exception as e:
            logger.error(f"Error parsing score for {key}: {e}")
            scores[key] = 5.0  # Default middle score if parsing fails

    # Calculate weighted total score
    weights = JOB_REQUIREMENTS["scoring_weights"]
    total_score = 0

    for i in range(1, 5):
        key = f"question_{i}"
        if key in scores:
            weight = weights[key]
            # Convert 0-10 score to 0-100 points with weight
            points = (scores[key] / 10) * (weight * 100)
            total_score += points

    # Determine tier
    if total_score >= 90:
        tier = 1
        logger.info(f"Candidate scored {total_score}% - TIER 1 (Auto-schedule)")
    elif total_score >= 75:
        tier = 2
        logger.info(f"Candidate scored {total_score}% - TIER 2 (Human approval needed)")
    else:
        tier = 3
        logger.info(f"Candidate scored {total_score}% - TIER 3 (Rejected)")

    return {
        "answers": answers,
        "scores": scores,
        "total_score": total_score,
        "tier": tier
    }


async def find_slots(state: RecruitmentState):
    """
    Find available interview slots using Google Calendar
    """
    logger.info("Finding available interview slots...")

    messages = state["messages"]
    last_message = messages[-1]
    tool_messages = []

    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for call in last_message.tool_calls:
            logger.info("Processing tool call: %s", call)
            tool_name = call.get("name")
            tool_id = call.get("id")
            args = call.get("args")

            if tool_name == "GOOGLECALENDAR_FIND_FREE_SLOTS":
                find_free_slots_tool = next(
                    (tool for tool in recruitment_tools if tool.name == tool_name), None
                )

                if find_free_slots_tool:
                    res = find_free_slots_tool.invoke(args)
                    tool_msg = ToolMessage(
                        name=tool_name,
                        content=str(res),
                        tool_call_id=tool_id
                    )
                    tool_messages.append(tool_msg)

    return {"messages": tool_messages}


async def hitl_approval(state: RecruitmentState) -> Command[Literal["auto_schedule", "reject"]]:
    """
    Human-in-the-loop: HR reviews candidate and approves/rejects
    """
    logger.info("Waiting for HR approval...")

    # Prepare candidate summary for HR
    candidate_info = {
        "name": state.get("candidate_name", "Unknown"),
        "email": state.get("candidate_email", "Unknown"),
        "score": state.get("total_score", 0),
        "tier": state.get("tier", 0),
        "answers": state.get("answers", {}),
        "scores": state.get("scores", {})
    }

    # Interrupt workflow and wait for HR decision
    hr_decision = interrupt({
        "message": "TIER 2 Candidate - HR Review Required",
        "candidate": candidate_info,
        "instructions": "Please review the candidate and type 'approve' to schedule interview or 'reject' to decline."
    })

    logger.info(f"HR Decision: {hr_decision}")

    if hr_decision and hr_decision.lower() == "approve":
        return Command(
            goto="auto_schedule",
            update={"hr_approved": True}
        )
    else:
        return Command(
            goto="reject",
            update={"hr_approved": False}
        )


def auto_schedule(state: RecruitmentState):
    """
    Automatically schedule interview for high-scoring candidates
    """
    logger.info("Auto-scheduling interview for qualified candidate...")

    return {
        "messages": [AIMessage(content=f"Congratulations! Your application is very strong (Score: {state.get('total_score', 0):.1f}%). Let me find available interview slots for you.")]
    }


def reject_candidate(state: RecruitmentState):
    """
    Politely reject candidates who don't meet requirements
    """
    logger.info("Rejecting candidate due to low score...")

    rejection_message = """Thank you for taking the time to apply for the QA Engineer position and for completing our screening questions.

After careful review of your responses, we regret to inform you that your qualifications don't align with our current requirements for this role.

We appreciate your interest in our company and encourage you to apply for future openings that may be a better match for your skills and experience.

We wish you the best in your job search!

Best regards,
The Recruitment Team"""

    return {
        "messages": [AIMessage(content=rejection_message)]
    }


def tier_2_waiting_message(state: RecruitmentState):
    """
    Send waiting message to TIER 2 candidates
    """
    logger.info("Sending review message to TIER 2 candidate...")

    waiting_message = """Thank you for completing our screening questions!

We're currently reviewing your application and will get back to you via email within 48 hours with next steps.

We appreciate your patience and interest in joining our team!

Best regards,
The Recruitment Team"""

    return {
        "messages": [AIMessage(content=waiting_message)]
    }

In [39]:
async def tools_condition(state: RecruitmentState) -> Literal["find_slots", "tools", "score_answers", "__end__"]:
    """
    Route based on tool calls or conversation state
    """
    messages = state["messages"]
    last_message = messages[-1]

    # Check if we have tool calls
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for call in last_message.tool_calls:
            tool_name = call.get("name")

            if tool_name == "GOOGLECALENDAR_FIND_FREE_SLOTS":
                return "find_slots"

        return "tools"

    # Check if screening is complete (look for 4 answers in conversation)
    human_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]

    # If we have enough responses and haven't scored yet
    if len(human_messages) >= 5 and state.get("total_score") is None:  # Name + 4 answers
        # Check if candidate provided email
        email_provided = any('@' in msg.content for msg in human_messages)
        if email_provided:
            return "score_answers"

    return "__end__"


def route_by_score(state: RecruitmentState) -> Literal["auto_schedule", "hitl_approval", "reject", "tier_2_waiting"]:
    """
    Route candidates based on their tier
    """
    tier = state.get("tier", 3)

    if tier == 1:
        logger.info("Routing to AUTO_SCHEDULE (Tier 1)")
        return "auto_schedule"
    elif tier == 2:
        logger.info("Routing to HITL_APPROVAL (Tier 2)")
        return "tier_2_waiting"  # First send waiting message
    else:
        logger.info("Routing to REJECT (Tier 3)")
        return "reject"


def check_hitl_needed(state: RecruitmentState) -> Literal["hitl_approval", "__end__"]:
    """
    Check if we need to wait for HITL after sending waiting message
    """
    if state.get("tier") == 2 and state.get("hr_approved") is None:
        return "hitl_approval"
    return "__end__"

In [40]:
# Create the workflow graph
workflow = StateGraph(RecruitmentState)

# Add all nodes
workflow.add_node("agent", call_model)
workflow.add_node("find_slots", find_slots)
workflow.add_node("tools", ToolNode(recruitment_tools_write))
workflow.add_node("score_answers", score_answers)
workflow.add_node("auto_schedule", auto_schedule)
workflow.add_node("hitl_approval", hitl_approval)
workflow.add_node("reject", reject_candidate)
workflow.add_node("tier_2_waiting", tier_2_waiting_message)

# Define edges
workflow.add_edge(START, "agent")

# Conditional edges from agent
workflow.add_conditional_edges(
    "agent",
    tools_condition,
    ["tools", "find_slots", "score_answers", END]
)

# Tools and find_slots go back to agent
workflow.add_edge("tools", "agent")
workflow.add_edge("find_slots", "agent")

# After scoring, route by tier
workflow.add_conditional_edges(
    "score_answers",
    route_by_score,
    ["auto_schedule", "tier_2_waiting", "reject"]
)

# Auto-schedule goes back to agent for calendar booking conversation
workflow.add_edge("auto_schedule", "agent")

# Tier 2 waiting message -> check if HITL needed
workflow.add_conditional_edges(
    "tier_2_waiting",
    check_hitl_needed,
    ["hitl_approval", END]
)

# HITL routes to either auto_schedule or reject based on HR decision
# (This is handled in the hitl_approval function using Command)

# Reject ends the conversation
workflow.add_edge("reject", END)


ValueError: The first argument must be a string or a callable with a __name__ for tool decorator. Got <class 'dict'>

In [ ]:
checkpointer = MemorySaver()

In [ ]:
app = workflow.compile(checkpointer=checkpointer)

logger.info("HR Recruitment Assistant workflow compiled successfully!")

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    logger.warning(f"Could not display graph: {e}")

In [ ]:
# Create a new conversation thread
config = {"configurable": {"thread_id": "candidate_001"}}

# Initial greeting
initial_state = {
    "messages": [HumanMessage(content="Hi, I'm interested in the QA Engineer position")]
}

print("=== CANDIDATE SCREENING STARTED ===\n")

async for chunk in app.astream(initial_state, config=config, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")


In [ ]:
# Candidate provides name and email
input_msg = {
    "messages": [HumanMessage(content="My name is Sarah Johnson and my email is sarah.johnson@email.com")]
}

async for chunk in app.astream(input_msg, config=config, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

In [ ]:
# Answer Question 1
print("\n=== ANSWERING QUESTION 1 ===\n")
input_msg = {
    "messages": [HumanMessage(content="I have 5 years of professional QA experience working in agile teams")]
}

async for chunk in app.astream(input_msg, config=config, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

# Answer Question 2
print("\n=== ANSWERING QUESTION 2 ===\n")
input_msg = {
    "messages": [HumanMessage(content="I've used Selenium, Cypress, Playwright for UI testing, and Postman and RestAssured for API testing. Also experienced with JUnit and TestNG frameworks.")]
}

async for chunk in app.astream(input_msg, config=config, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

# Answer Question 3
print("\n=== ANSWERING QUESTION 3 ===\n")
input_msg = {
    "messages": [HumanMessage(content="I found a critical race condition bug where the checkout process would fail intermittently under load. I documented the exact steps to reproduce, included network logs, database states, and worked with developers to identify it was a transaction isolation issue. We fixed it by implementing proper locking mechanisms.")]
}

async for chunk in app.astream(input_msg, config=config, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

# Answer Question 4
print("\n=== ANSWERING QUESTION 4 ===\n")
input_msg = {
    "messages": [HumanMessage(content="Yes, very comfortable! I've designed and executed comprehensive API test suites including positive/negative scenarios, boundary testing, security testing, and performance testing. I'm familiar with REST and GraphQL APIs.")]
}

async for chunk in app.astream(input_msg, config=config, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

print("\n=== Check the score and tier assignment above ===\n")

In [ ]:
# Create new thread for Tier 2 candidate
config_tier2 = {"configurable": {"thread_id": "candidate_tier2"}}

print("\n\n=== TIER 2 CANDIDATE EXAMPLE ===\n")

# Start conversation
initial_state = {
    "messages": [HumanMessage(content="Hi, I want to apply for QA Engineer")]
}

async for chunk in app.astream(initial_state, config=config_tier2, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

# Provide details
input_msg = {
    "messages": [HumanMessage(content="I'm John Smith, email is john.smith@email.com")]
}

async for chunk in app.astream(input_msg, config=config_tier2, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

# Moderate answers (will score 75-89%)
answers_tier2 = [
    "I have about 3 years of QA experience",
    "I've used Selenium and some manual testing tools",
    "I once found a bug in the login page where special characters caused issues. I reported it to the team and they fixed it.",
    "Yes, I've done some API testing with Postman"
]

for answer in answers_tier2:
    input_msg = {"messages": [HumanMessage(content=answer)]}
    async for chunk in app.astream(input_msg, config=config_tier2, stream_mode="values"):
        response_message = chunk["messages"][-1]
        if hasattr(response_message, 'content') and response_message.content:
            print(f"Assistant: {response_message.content}\n")

print("\n=== At this point, the workflow is INTERRUPTED waiting for HR approval ===")
print("=== Check the interrupt payload above ===\n")

# To resume after HR approval:
# async for chunk in app.astream(Command(resume="approve"), config=config_tier2, stream_mode="values"):
#     ...

In [ ]:
# Create new thread for Tier 3 candidate
config_tier3 = {"configurable": {"thread_id": "candidate_tier3"}}

print("\n\n=== TIER 3 CANDIDATE EXAMPLE (Rejection) ===\n")

# Start conversation
initial_state = {
    "messages": [HumanMessage(content="Hello, applying for QA position")]
}

async for chunk in app.astream(initial_state, config=config_tier3, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

# Provide details
input_msg = {
    "messages": [HumanMessage(content="Bob Brown, bob@email.com")]
}

async for chunk in app.astream(input_msg, config=config_tier3, stream_mode="values"):
    response_message = chunk["messages"][-1]
    if hasattr(response_message, 'content') and response_message.content:
        print(f"Assistant: {response_message.content}\n")

# Weak answers (will score <75%)
answers_tier3 = [
    "I'm new to QA, just started learning",
    "I don't know many tools yet",
    "Haven't found any bugs yet",
    "Not really familiar with API testing"
]

for answer in answers_tier3:
    input_msg = {"messages": [HumanMessage(content=answer)]}
    async for chunk in app.astream(input_msg, config=config_tier3, stream_mode="values"):
        response_message = chunk["messages"][-1]
        if hasattr(response_message, 'content') and response_message.content:
            print(f"Assistant: {response_message.content}\n")

print("\n=== Candidate automatically rejected due to low score ===\n")